In [1]:
import os,sys,string, time
from array import array
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
from matplotlib.patches import Rectangle
import pickle
import functools
from importlib import reload

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print ('Success')


Success


## Getting runs in goodrun list

In [16]:
def Get_runs_in_goodruns(root_file_path, goodruns_list):
    """
    Input .root file and \"goodrun\" list.
    Return lists of runs in list and runs not in list.
    """
    root_file = uproot3.open(root_file_path)['nuselection/NeutrinoSelectionFilter']
    df_file = root_file.pandas.df(['run'], flatten=False)
    
    with open(goodruns_list, "r") as file:
        goodruns = [int(line.strip()) for line in file]
        
    unique_values = list(set(goodruns))
    unique_file_subruns = list(set(np.array(df_file['run'])))
    
    runs_in_goodruns, runs_not_in_goodruns = [], []
    
    for run in unique_file_subruns:
        if run in unique_values: runs_in_goodruns.append(run)
        if run not in unique_values: runs_not_in_goodruns.append(run)
        
    return runs_in_goodruns, runs_not_in_goodruns

def Get_runs_in_horncurrent(root_file_path, run_range):
    """
    Input root file and a list for the run range.
    """
    root_file = uproot3.open(root_file_path)['nuselection/NeutrinoSelectionFilter']
    df_file = root_file.pandas.df(['run'], flatten=False)
    
    runs_in_range = np.arange(run_range[0], run_range[-1]+1)
    
    unique_file_runs = list(set(np.array(df_file['run'])))
    
    runs_in_goodruns, runs_not_in_goodruns = [], []
    
    for run in unique_file_runs:
        if run in runs_in_range: runs_in_goodruns.append(run)
        if run not in runs_in_range: runs_not_in_goodruns.append(run)
        
    return runs_in_goodruns, runs_not_in_goodruns

def Filter_df_runs(df, goodruns_list):
    """
    Input dataframe and goodruns list.
    Return the filtered dataframe, with only the goodruns included.
    """
    filtered = df.loc[(df['run'].isin(goodruns_list))]
    
    return filtered
    
def Return_runs_subruns(df):
    """
    Input filtered df.
    Return lists of runs and subruns.
    """
    # if pd.Series(['run', 'sub']).isin(df.columns).all():
    #     run_sub_list = []
    #     for entry in df.index: #Looping over all events in the dataframe
    #         run_sub = str(df['run'][entry]) + "_" + str(df['sub'][entry])
    #         run_sub_list.append(run_sub)
    #     unique_run_subrun = list(set(run_sub_list))
    #     return df.copy()
    # else:
    #     print("Dataframe needs \"run\", \"sub\" columns.")
    #     return 0
    placeholder=df.drop_duplicates(subset=["run","sub"]).copy()
    runs_list = placeholder["run"]
    subruns_list = placeholder["sub"]
    
    return runs_list, subruns_list

def Save_runs_subruns_as_txt(runs_list, subruns_list, savename):
    """
    Input list of runs and subruns.
    Saves runs subruns file as savename.
    """
    with open(savename, "w") as file:
        for val1, val2 in zip(runs_list, subruns_list):
            file.write(str(val1) + " " + str(val2) + "\n")
    

In [17]:
#FHC: 14 Nov 2016 - 20 Feb 2017 | run 8784 - 10139
#RHC: 20 Feb 2017 - 7 July 2017 | run 10140 - 11949
FHC_runs = [8784, 10139]
RHC_runs = [10140, 11949]

In [18]:
runs_in_range = np.arange(FHC_runs[0], FHC_runs[-1]+1)


In [19]:
FHC_r2a, not_FHC_r2a = Get_runs_in_horncurrent("../NuMI_data/neutrinoselection_filt_run2a_beamon.root", FHC_runs)
RHC_r2b, not_RHC_r2b = Get_runs_in_horncurrent("../NuMI_data/neutrinoselection_filt_run2a_beamon.root", RHC_runs)

In [21]:
print(len(FHC_r2a), len(not_FHC_r2a))
print(len(RHC_r2b), len(not_RHC_r2b))

684 502
478 708


In [ ]:
runs_in_goodruns_2a, not_in_goodruns_2a = Get_runs_in_goodruns("../NuMI_data/neutrinoselection_filt_run2a_beamoff.root", "../NuMI_data/run2_goodruns.txt")
runs_in_goodruns_2b, not_in_goodruns_2b = Get_runs_in_goodruns("../NuMI_data/neutrinoselection_filt_run2b_beamoff.root", "../NuMI_data/run2_goodruns.txt")

print(len(runs_in_goodruns_2a), len(not_in_goodruns_2a))
print(len(runs_in_goodruns_2b), len(not_in_goodruns_2b))

print(type(runs_in_goodruns_2a))

In [ ]:
r2a_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2a_beamon.root")['nuselection/NeutrinoSelectionFilter']
full_run2a_df = r2a_file.pandas.df(['run', 'sub'], flatten=False)
r2b_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2b_beamon.root")['nuselection/NeutrinoSelectionFilter']
full_run2b_df = r2b_file.pandas.df(['run', 'sub'], flatten=False)

print("run2a beamon length: " + str(len(full_run2a_df)))
print("run2b beamon length: " + str(len(full_run2b_df)))

all_run2a_runs = list(set(np.array(full_run2a_df['run'])))
all_run2b_runs = list(set(np.array(full_run2b_df['run'])))

print("run2a beamon all runs: " + str(len(all_run2a_runs)))
print("run2b beamon all runs: " + str(len(all_run2b_runs)))

run2a_filtered = Filter_df_runs(full_run2a_df, all_run2a_runs)
run2b_filtered = Filter_df_runs(full_run2b_df, all_run2b_runs)

print("run2a beamon length: " + str(len(run2a_filtered)))
print("run2b beamon length: " + str(len(run2b_filtered)))

run2a_runs, run2a_subruns = Return_runs_subruns(run2a_filtered)
run2b_runs, run2b_subruns = Return_runs_subruns(run2b_filtered)

print("run2a")
print(len(run2a_runs), len(run2a_subruns))
print("run2b")
print(len(run2b_runs), len(run2b_subruns))

In [ ]:
Save_runs_subruns_as_txt(run2a_runs, run2a_subruns, "../NuMI_data/FULL_beamon_run2a_run_sub.txt")
Save_runs_subruns_as_txt(run2b_runs, run2b_subruns, "../NuMI_data/FULL_beamon_run2b_run_sub.txt")

In [ ]:
r2a_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2a_beamoff.root")['nuselection/NeutrinoSelectionFilter']
full_run2a_df = r2a_file.pandas.df(['run', 'sub'], flatten=False)
r2b_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2b_beamoff.root")['nuselection/NeutrinoSelectionFilter']
full_run2b_df = r2b_file.pandas.df(['run', 'sub'], flatten=False)

print("run2a beamon length: " + str(len(full_run2a_df)))
print("run2b beamon length: " + str(len(full_run2b_df)))

all_run2a_runs = list(set(np.array(full_run2a_df['run'])))
all_run2b_runs = list(set(np.array(full_run2b_df['run'])))

print("run2a beamoff all runs: " + str(len(all_run2a_runs)))
print("run2b beamoff all runs: " + str(len(all_run2b_runs)))

run2a_filtered = Filter_df_runs(full_run2a_df, all_run2a_runs)
run2b_filtered = Filter_df_runs(full_run2b_df, all_run2b_runs)

print("run2a beamon length: " + str(len(run2a_filtered)))
print("run2b beamon length: " + str(len(run2b_filtered)))

run2a_runs, run2a_subruns = Return_runs_subruns(run2a_filtered)
run2b_runs, run2b_subruns = Return_runs_subruns(run2b_filtered)

print("run2a")
print(len(run2a_runs), len(run2a_subruns))
print("run2b")
print(len(run2b_runs), len(run2b_subruns))

In [ ]:
Save_runs_subruns_as_txt(run2a_runs, run2a_subruns, "../NuMI_data/FULL_beamoff_run2a_run_sub.txt")
Save_runs_subruns_as_txt(run2b_runs, run2b_subruns, "../NuMI_data/FULL_beamoff_run2b_run_sub.txt")

In [ ]:
run2a_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2a_beamoff.root")['nuselection/NeutrinoSelectionFilter']
run2b_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2b_beamoff.root")['nuselection/NeutrinoSelectionFilter']

run2a_df = run2a_file.pandas.df(['run', 'sub', 'evt'], flatten=False)
run2b_df = run2b_file.pandas.df(['run', 'sub', 'evt'], flatten=False)

run2a_filtered = Filter_df_runs(run2a_df, runs_in_goodruns_2a)
run2b_filtered = Filter_df_runs(run2b_df, runs_in_goodruns_2b)

run2a_runs, run2a_subruns = Return_runs_subruns(run2a_filtered)
run2b_runs, run2b_subruns = Return_runs_subruns(run2b_filtered)

print(len(run2a_runs), len(run2a_subruns))
print(len(run2b_runs), len(run2b_subruns))


In [ ]:
Save_runs_subruns_as_txt(run2a_runs, run2a_subruns, "../NuMI_data/beamoff_run2a_run_sub.txt")
Save_runs_subruns_as_txt(run2b_runs, run2b_subruns, "../NuMI_data/beamoff_run2b_run_sub.txt")

In [ ]:
file_locs = Functions.Get_all_sample_locs(Params)

In [ ]:
root_file_loc = file_locs["beamgood"]
print(root_file_loc)

In [ ]:
file_path = "../NuMI_data/run2_goodruns.txt"  

with open(file_path, "r") as file:
    values = [int(line.strip()) for line in file]
    
root_file = uproot3.open(root_file_loc)['nuselection/NeutrinoSelectionFilter']

df_file = root_file.pandas.df(['run', 'sub', 'evt'], flatten=False)


In [ ]:
unique_values = list(set(values))
unique_file_subruns = list(set(np.array(df_file['run'])))

runs_in_goodruns = []
runs_not_in_goodruns = []

for run in unique_file_subruns:
    if run in unique_values: runs_in_goodruns.append(run)
    if run not in unique_values: runs_not_in_goodruns.append(run)
    
print("Total runs in run2a file is " + str(len(unique_file_subruns)))
print("runs in goodrun list: " + str(len(runs_in_goodruns)))
print("runs NOT in goodrun list: " + str(len(runs_not_in_goodruns)))

In [ ]:
file_path = "../NuMI_data/run2_goodruns.txt"  

with open(file_path, "r") as file:
    values = [int(line.strip()) for line in file]
    
root_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2b_beamon.root")['nuselection/NeutrinoSelectionFilter']

df_file = root_file.pandas.df(['run', 'sub', 'evt'], flatten=False)

In [ ]:
unique_values = list(set(values))
unique_file_subruns = list(set(np.array(df_file['run'])))

runs_in_goodruns = []
runs_not_in_goodruns = []

for run in unique_file_subruns:
    if run in unique_values: runs_in_goodruns.append(run)
    if run not in unique_values: runs_not_in_goodruns.append(run)
    
print("Total runs in run2b file is ")
print("runs in goodrun list: " + str(len(runs_in_goodruns)))
print("runs NOT in goodrun list: " + str(len(runs_not_in_goodruns)))

## End of code